In [1]:
import os
import shutil
from IPython.display import clear_output
from timeit import default_timer as timer
from tqdm import tqdm
import datetime


In [5]:
class DTU:
    def __init__(self):
        self.source=""
        self.destination=""
        self.mediafolder="D:\\sourceDir"
        self.drives=os.listdir(self.mediafolder)
        #self.destFolder="USB Transfer"
        
    def showMenu(self):
        print("-----MENU-----")
        for i in range(1,6):
            if i == 1:
                print("[1] Show all sources\n")

            elif i== 2:
                print("[2] Select source\n")

            elif i== 3:
                print("[3] Select Destination\n")

            elif i== 4:
                print("[4] Transfer content\n")

            elif i== 5:
                print("[5] Exit\n")

            else:
                break

        choice = int(input("Please select an option: "))
        print("--------------")
        return choice

    def choiceSelected(self,choice):
        try:
            
            #Show all sources
            if choice == 1:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)
                return

            #Select source
            elif choice == 2:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)

                select = self.askChoice("Select a source: ")
                self.source=self.drives[select-1]
                print("\nSource selected!\n")
                return

            #Select dest
            elif choice == 3:
                print("Availabe destinations:\n")
                for i,source in enumerate(self.drives):
                    if source==self.source:
                        continue
                    print(i+1,source)

                select = self.askChoice("Select a destination: ")
                self.destination=self.drives[select-1]
                print("\nDestination selected!\n")
                return

            #Transfer content
            elif choice == 4:
                print("[1] Transfer all content\n","\n[2] Select content to transfer")
                ch=self.askChoice("\nChoose an option: ")
                if ch == 1:
                    start = timer()
                    files = os.listdir(os.path.join(self.mediafolder,self.source))
                    destinationDir = os.path.join(self.mediafolder,self.destination)
                    sourceDir = os.path.join(self.mediafolder,self.source)
                    # paste commented code here
                    for f in tqdm(range(len(files)),unit='KB'):
                        if os.path.isdir(os.path.join(sourceDir,files[f])):
                            shutil.copytree(os.path.join(sourceDir,files[f]),os.path.join(destinationDir,str(files[f])))
                    end = timer()
                    print("Time-Elapsed : " ,end - start,"sec.")

                elif ch == 2:
                    pass
                
                return

            elif choice == 5:
                pass

        except Exception as e:
            print("ERROR: ",e)
        
    def askChoice(self,string):
        return int(input(string))
if __name__ == "__main__":
    print("Welcome to DTU")
    dtu=DTU()
    while(True):
        ch = dtu.showMenu()

        #Exit condition
        if ch not in [1,2,3,4]:
            print("Bye for now!")
            break

        #On choice selected
        dtu.choiceSelected(ch)
        input("press [ENTER] to continue")
        clear_output()

-----MENU-----
[1] Show all sources

[2] Select source

[3] Select Destination

[4] Transfer content

[5] Exit

Please select an option: 5
--------------
Bye for now!
